# **Perspectiva General de Mercado - DatAlpine**
*Filtro de tipo de mercado*

In [1]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
import numpy as np

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()
# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    if nombre.startswith("df_") and "alfa" in nombre and "pachuca" in nombre and '2024' in nombre
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_alfa_abril_2024_pachuca',
 'df_alfa_agosto_2024_pachuca',
 'df_alfa_febrero_2024_pachuca',
 'df_alfa_julio_2024_pachuca',
 'df_alfa_junio_2024_pachuca',
 'df_alfa_marzo_2024_pachuca',
 'df_alfa_mayo_2024_pachuca']

In [2]:
# Iterar sobre cada DataFrame en la lista filtrada
for nombre_df in nombres_df_filtrados:
    # Obtener el DataFrame usando globals()
    df = globals()[nombre_df]
    df.rename(columns={'Categoria':'categoria','Category':'categoria','Propiedad':'propiedad','Precio':'precio','Metros_total':'metros_total','m_total':'metros_total','m_construido':'metros_construido','Metros_construido':'metros_construido','m2_contruido':'metros_construido','Estacionamientos':'estacionamiento','Recamaras':'recamaras','Banos':'baño','Medio_banos':'medio_baño','Banos_Total':'baño_total','Precio_M2_Construido':'precio_m2_construido','precio_m2_contruido':'precio_m2_construido'}, inplace=True)
        # Asignar el DataFrame modificado de nuevo a la variable global
    globals()[nombre_df] = df 
print("Columnas renombradas en los DataFrames filtrados.")

Columnas renombradas en los DataFrames filtrados.


In [3]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
for nombre_df in nombres_df_filtrados:
    # Seleccionar las columnas 'id' y 'categoria'
    segment_df = globals()[nombre_df][['id','Tipo','categoria','propiedad','precio','metros_total','metros_construido', 'precio_m2_construido','estacionamiento','recamaras', 'baño', 'medio_baño', 'baño_total']]
    # Añadir el DataFrame a la lista
    dataframes_list.append(segment_df)

In [4]:
df_total = pd.concat(dataframes_list, ignore_index=True)

In [5]:
df_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11650 entries, 0 to 11649
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    11650 non-null  int64  
 1   Tipo                  11650 non-null  object 
 2   categoria             11650 non-null  object 
 3   propiedad             11650 non-null  object 
 4   precio                11650 non-null  int64  
 5   metros_total          11650 non-null  int64  
 6   metros_construido     11650 non-null  int64  
 7   precio_m2_construido  11650 non-null  float64
 8   estacionamiento       11650 non-null  int64  
 9   recamaras             11650 non-null  int64  
 10  baño                  11650 non-null  int64  
 11  medio_baño            11650 non-null  int64  
 12  baño_total            11650 non-null  float64
dtypes: float64(2), int64(8), object(3)
memory usage: 1.2+ MB


In [6]:
# Definición de rangos de precio
rangos_precio = {
    "E1": (0, 500000),
    "E2": (500001, 750000),
    "E3": (750001, 1000000),
    "D1": (1000001, 1250000),
    "D2": (1250001, 1500000),
    "D3": (1500001, 1750000),
    "C1": (1750001, 2000000),
    "C2": (2000001, 2250000),
    "C3": (2250001, 2500000),
    "B1": (2500001, 2750000),
    "B2": (2750001, 3000000),
    "B3": (3000001, 3250000),
    "A1": (3250001, 3500000),
    "A2": (3500001, 3750000),
    "A3": (3750001, 4000000),
    "S1": (4000001, 6000000),
    "S2": (6000001, 8000000),
    "S3": (8000001, 12000000),
    "L1": (12000001, 14000000),
    "L2": (14000001, 16000000),
    "L3": (16000001, 18000000),
    "L+": (18000001, 22000000),
    "ELITE": (22000001, float('inf')),
}

# Función para asignar la categoría según el precio
def asignar_categoria(precio):
    for categoria, (limite_inferior, limite_superior) in rangos_precio.items():
        if limite_inferior <= precio <= limite_superior:  # Cambiado a <=
            return categoria
    return None  # Retornar None para manejarlo después

# Limpieza del DataFrame
df_total = df_total.drop(columns=['categoria'], errors='ignore')

# Convertir 'precio' a numérico y manejar errores
df_total['precio'] = pd.to_numeric(df_total['precio'], errors='coerce')
df_total = df_total.dropna(subset=['precio'])  # Eliminar filas con precios NaN

# Asignar categorías
df_total['categoria'] = df_total['precio'].apply(asignar_categoria)

# Verificar categorías inválidas
categorias_invalidas = df_total[df_total['categoria'].isnull()]
if not categorias_invalidas.empty:
    print("Precios sin categoría:")
    print(categorias_invalidas[['precio']])

In [7]:
df_total.loc[:, 'segmento'] = df_total['categoria'].apply(lambda x: 'ELITE' if x == 'ELITE' else x[0])

In [8]:
import numpy as np 
condition = (df_total['baño_total'] % 1 == 0.5) # Valores que contienen .5    
# Convertir al número entero anterior
df_total.loc[condition, 'baño_total'] = np.floor(df_total['baño_total']) 

# Limitar los valores  
df_total['baño_total'] = df_total['baño_total'].clip(lower=1) #,upper=10
df_total['estacionamiento'] = df_total['estacionamiento'].clip(lower=1) #upper=8
df_total['recamaras'] = df_total['recamaras'].clip(lower=1)#upper=8 

df_total = df_total[df_total['precio'] != 0]
df_total = df_total[df_total['metros_total'] != 0]
df_total = df_total[df_total['metros_construido'] != 0]

#df_total = df_total[~df_total['categoria'].str.contains('L')]

- Casa promedio
- Casa Mediana
- Precio mercado

- m2 Terreno
- M2 Construcción
- Precio m2 construcción

In [44]:
df_total['Tipo'].unique()

array(['Casa', 'Casa En Fraccionamiento', 'Casa En Condominio',
       'Departamento', 'Penthouse', 'Estudio', 'Condominio Horizontal',
       'Duplex', 'Casa ', 'Daoplex'], dtype=object)

In [45]:
casa_df = df_total[df_total['Tipo'].isin(['Casa', 'Casa En Fraccionamiento', 'Casa En Condominio', 'Casa Duplex', 'Casa'])]
depa_df = df_total[df_total['Tipo'].isin(['Departamento', 'Duplex', 'Condominio Horizontal', 'Penthouse', 'Estudio', 'Daoplex'])]

# Casas

### Revisión de m2

In [46]:
casa_df = casa_df[casa_df['metros_total'] >= 50]
casa_df = casa_df[casa_df['metros_construido'] >= 50]

In [47]:
# metros_total en casas
precios = casa_df['metros_total'].unique()
precios = np.sort(precios)
precios

array([   50,    51,    53,    54,    55,    56,    58,    59,    60,
          61,    62,    63,    64,    65,    66,    67,    69,    70,
          74,    75,    76,    79,    80,    82,    83,    84,    85,
          86,    87,    88,    89,    90,    92,    93,    94,    95,
          96,    97,    98,    99,   100,   101,   102,   103,   104,
         105,   106,   107,   108,   109,   110,   111,   112,   113,
         114,   115,   116,   117,   118,   119,   120,   121,   122,
         123,   124,   125,   126,   127,   128,   129,   130,   131,
         132,   133,   134,   135,   136,   137,   138,   139,   140,
         141,   142,   143,   144,   145,   146,   147,   148,   149,
         150,   151,   152,   154,   155,   156,   157,   158,   159,
         160,   161,   162,   163,   164,   165,   166,   167,   168,
         169,   170,   172,   173,   174,   175,   176,   177,   178,
         179,   180,   181,   182,   183,   184,   185,   186,   187,
         188,   189,

In [48]:
# metros_construido en casas
precios = casa_df['metros_construido'].unique()
precios = np.sort(precios)
precios

array([   50,    51,    52,    53,    54,    55,    56,    57,    58,
          60,    61,    62,    64,    65,    66,    67,    68,    69,
          70,    71,    72,    73,    74,    75,    76,    77,    79,
          80,    81,    82,    83,    84,    85,    86,    87,    88,
          89,    90,    91,    92,    93,    94,    95,    96,    97,
          98,    99,   100,   101,   102,   103,   104,   105,   106,
         107,   108,   109,   110,   111,   112,   113,   114,   115,
         116,   117,   118,   119,   120,   121,   122,   123,   124,
         125,   126,   127,   128,   129,   130,   131,   132,   133,
         134,   135,   136,   137,   138,   139,   140,   141,   142,
         143,   144,   145,   146,   147,   148,   149,   150,   151,
         152,   153,   154,   155,   156,   157,   158,   159,   160,
         161,   162,   163,   164,   165,   166,   167,   168,   169,
         170,   171,   172,   173,   174,   175,   176,   177,   178,
         180,   181,

CAMBIOS DE M2

In [63]:
printtt = df_total[df_total['metros_total'] == 2451]
printtt

,id,Tipo,propiedad,precio,metros_total,metros_construido,precio_m2_construido,estacionamiento,recamaras,baño,medio_baño,baño_total,categoria,segmento
2559,1013,Casa,REAL DE LAS ESMERALDAS CASA VENTA PACHUCA HIDALGO,21000000,2451,495,42424.24,1,3,4,1,4.0,L+,L
2968,16,Casa,REAL DE LAS ESMERALDAS CASA VENTA PACHUCA HIDALGO,21000000,2451,495,42424.24,1,3,4,0,4.0,L+,L
6181,1336,Casa,REAL DE LAS ESMERALDAS CASA VENTA PACHUCA HIDALGO,21000000,2451,495,42424.24,5,3,4,1,4.0,L+,L
8219,1611,Casa,REAL DE LAS ESMERALDAS CASA VENTA PACHUCA HIDALGO,21000000,2451,495,42424.24,4,3,1,0,1.0,L+,L
8252,16,Casa,real de las esmeraldas casa venta pachuca hidalgo,21000000,2451,495,42424.24,2,3,1,0,1.0,L+,L
10512,669,Casa,real de las esmeraldas casa venta pachuca hidalgo,21000000,2451,495,42424.24,2,3,1,0,1.0,L+,L


## Casa promedio

In [16]:
#casa_df.loc[:, 'segmento'] = casa_df['categoria'].str[0]
grupo_promedios = round(casa_df.groupby('segmento')[['estacionamiento', 'recamaras', 'baño_total']].mean(),0)
grupo_promedios.astype(int)
# Renombrar las columnas para mayor claridad
grupo_promedios.rename(columns={
    'estacionamiento': 'Estacionamientos',
    'recamaras': 'Recámaras',
    'baño_total': 'Baños',
}, inplace=True)

# Calcular el promedio de estacionamientos, recámaras y baños en general (sin segmento específico)
promedios_generales = round(casa_df[['estacionamiento', 'recamaras', 'baño_total']].mean(),0)
promedios_generales = promedios_generales.astype(int)
# Renombrar las columnas para mayor claridad
promedios_generales.rename(index={
    'estacionamiento': 'Estacionamientos',
    'recamaras': 'Recámaras',
    'baño_total': 'Baños',
}, inplace=True)
# Agregar los promedios generales al dataframe de grupo_promedios
promedios_generales = pd.DataFrame(promedios_generales).T
promedios_generales['segmento'] = 'general'

casa_promedio = pd.concat([grupo_promedios, promedios_generales.set_index('segmento')], ignore_index=False)

#print(grupo_promedios)
#print(promedios_generales)
print(casa_promedio)

# Convertir el dataframe a JSON
grupo_promedios_json = casa_promedio.to_json(orient='index')

# Guardar el JSON en un archivo
with open('assets/tablas/grupo_casa_promedio.json', 'w') as f:
    f.write(grupo_promedios_json)

          Estacionamientos  Recámaras  Baños
segmento                                    
A                      2.0        4.0    2.0
B                      2.0        3.0    2.0
C                      2.0        3.0    2.0
D                      2.0        3.0    2.0
E                      2.0        3.0    1.0
ELITE                  2.0        4.0    4.0
L                      2.0        4.0    3.0
S                      2.0        4.0    3.0
general                2.0        3.0    2.0


## Precio Mercado

In [17]:
#casa_df.loc[:, 'segmento'] = casa_df['categoria'].str[0].astype(str)
orden_segmentos = ['E', 'D', 'C', 'B', 'A', 'S', 'L','ELITE']#
grupo_descripcion_casa = casa_df.groupby('segmento')['precio'].describe().reset_index()
grupo_descripcion_casa = grupo_descripcion_casa.loc[:, ['segmento','min', 'max','mean']] #Columnas deseadas
grupo_descripcion_casa.rename(columns={
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)
# Ordenar el dataframe según el orden específico de segmentos
ordenado = grupo_descripcion_casa.set_index('segmento').reindex(orden_segmentos).reset_index()
ordenado

,segmento,Mínimo,Máximo,Promedio
0,E,4800.0,1000000.0,7.003255e+05
1,D,1005000.0,1750000.0,1.409342e+06
2,C,1750050.0,2500000.0,2.140685e+06
3,B,2520000.0,3250000.0,2.895381e+06
4,A,3255000.0,4000000.0,3.613227e+06
5,S,4029000.0,12000000.0,6.100476e+06
6,L,12100000.0,22000000.0,1.669303e+07
7,ELITE,22450000.0,90000000.0,2.611848e+07


In [18]:
# Función para formatear valores en formato monetario
def formato_monetario(valor):
    return f"${valor:,.2f}"  # Formato con $ y 2 decimales, separador de miles

# Aplicar formato en m2 a los valores, excluyendo la columna 'segmento'
for i in range(1, 4):  # 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio'
    grupo_descripcion_casa.iloc[:, i] = grupo_descripcion_casa.iloc[:, i].astype(float).apply(formato_monetario)
grupo_descripcion_casa 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_11740\3321038986.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0     $3,255,000.00
1     $2,520,000.00
2     $1,750,050.00
3     $1,005,000.00
4         $4,800.00
5    $22,450,000.00
6    $12,100,000.00
7     $4,029,000.00
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_descripcion_casa.iloc[:, i] = grupo_descripcion_casa.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_11740\3321038986.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0     $4,000,000.00
1     $3,250,000.00
2     $2,500,000.00
3     $1,750,000.00
4     $1,000,000.00
5    $90,000,000.00
6    $22,000,000.00
7    $12,000,000.00
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicit

,segmento,Mínimo,Máximo,Promedio
0,A,"$3,255,000.00","$4,000,000.00","$3,613,226.99"
1,B,"$2,520,000.00","$3,250,000.00","$2,895,380.98"
2,C,"$1,750,050.00","$2,500,000.00","$2,140,685.10"
3,D,"$1,005,000.00","$1,750,000.00","$1,409,342.37"
4,E,"$4,800.00","$1,000,000.00","$700,325.47"
5,ELITE,"$22,450,000.00","$90,000,000.00","$26,118,483.87"
6,L,"$12,100,000.00","$22,000,000.00","$16,693,029.66"
7,S,"$4,029,000.00","$12,000,000.00","$6,100,476.28"


In [19]:
# Filtrar los datos según las condiciones especificadas
#filtrado_df = df_total[(~df_total['metros_construido'].isin([0, 1, 2, 3, 4, 5])) & (df_total['precio_m2_construido'] >= 280000)]

# Calcular las estadísticas descriptivas
todo_descripcion = casa_df['precio'].describe()[['min', 'max', 'mean']].to_frame().T
todo_descripcion['segmento'] = 'general' # Agregar la columna de segmento
todo_descripcion.rename(columns={
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)

# Calcular el percentil 95%
#percentil_95 = filtrado_df['precio_m2_construido'].quantile(0.95)
# Reemplazar el valor máximo con el percentil 95%
#todo_descripcion['Máximo'] = percentil_95

# Aplicar formato monetario a los valores
def formato_monetario(valor):
    return f"${valor:,.2f}"  # Formato monetario con dos decimales

for i in range(3):  # 0, 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
print(todo_descripcion)

           Mínimo          Máximo       Promedio segmento
precio  $4,800.00  $90,000,000.00  $3,238,487.38  general


C:\Users\yoe11\AppData\Local\Temp\ipykernel_11740\2555968983.py:23: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'precio    $4,800.00
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_11740\2555968983.py:23: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'precio    $90,000,000.00
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_11740\2555968983.py:23: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise

In [20]:
resultado_final = pd.concat([grupo_descripcion_casa, todo_descripcion], ignore_index=True)
print(resultado_final)

  segmento          Mínimo          Máximo        Promedio
0        A   $3,255,000.00   $4,000,000.00   $3,613,226.99
1        B   $2,520,000.00   $3,250,000.00   $2,895,380.98
2        C   $1,750,050.00   $2,500,000.00   $2,140,685.10
3        D   $1,005,000.00   $1,750,000.00   $1,409,342.37
4        E       $4,800.00   $1,000,000.00     $700,325.47
5    ELITE  $22,450,000.00  $90,000,000.00  $26,118,483.87
6        L  $12,100,000.00  $22,000,000.00  $16,693,029.66
7        S   $4,029,000.00  $12,000,000.00   $6,100,476.28
8  general       $4,800.00  $90,000,000.00   $3,238,487.38


In [21]:
orden_segmentos = ['E', 'D', 'C', 'B', 'A', 'S', 'L','ELITE','general']#
ordenado = resultado_final.set_index('segmento').reindex(orden_segmentos).reset_index()
ordenado

,segmento,Mínimo,Máximo,Promedio
0,E,"$4,800.00","$1,000,000.00","$700,325.47"
1,D,"$1,005,000.00","$1,750,000.00","$1,409,342.37"
2,C,"$1,750,050.00","$2,500,000.00","$2,140,685.10"
3,B,"$2,520,000.00","$3,250,000.00","$2,895,380.98"
4,A,"$3,255,000.00","$4,000,000.00","$3,613,226.99"
5,S,"$4,029,000.00","$12,000,000.00","$6,100,476.28"
6,L,"$12,100,000.00","$22,000,000.00","$16,693,029.66"
7,ELITE,"$22,450,000.00","$90,000,000.00","$26,118,483.87"
8,general,"$4,800.00","$90,000,000.00","$3,238,487.38"


In [22]:
# Guardar el JSON en un archivo
resultados_finales_desc_json = resultado_final.to_json(orient='index')
with open('assets/tablas/grupo_precio_mercado_casa.json', 'w') as f:
    f.write(resultados_finales_desc_json)

## M2 Terreno

In [23]:
#casa_df.loc[:, 'segmento'] = casa_df['categoria'].str[0].astype(str)
orden_segmentos = ['E', 'D', 'C', 'B', 'A', 'S', 'L','ELITE']#
grupo_descripcion_casa = casa_df.groupby('segmento')['metros_total'].describe().reset_index()
grupo_descripcion_casa = grupo_descripcion_casa.loc[:, ['segmento','min', 'max','mean']] #Columnas deseadas
grupo_descripcion_casa.rename(columns={
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)
# Ordenar el dataframe según el orden específico de segmentos
ordenado = grupo_descripcion_casa.set_index('segmento').reindex(orden_segmentos).reset_index()
ordenado

,segmento,Mínimo,Máximo,Promedio
0,E,42.0,5440.0,143.737817
1,D,51.0,8315.0,124.031565
2,C,63.0,14079.0,150.627694
3,B,70.0,15794.0,174.299733
4,A,90.0,19386.0,200.187903
5,S,100.0,32240.0,270.628602
6,L,245.0,2451.0,689.322034
7,ELITE,500.0,10000.0,1086.532258


In [24]:
# Función para formatear valores en formato monetario por m2
def formato_monetario_m2(valor):
    return f"{valor:,.2f} m²"  # Formato con $, 2 decimales, separador de miles y unidad m²

# Aplicar formato en m2 a los valores, excluyendo la columna 'segmento'
for i in range(1, 4):  # 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio'
    grupo_descripcion_casa.iloc[:, i] = grupo_descripcion_casa.iloc[:, i].astype(float).apply(formato_monetario_m2)
grupo_descripcion_casa 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_11740\3382106320.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0     90.00 m²
1     70.00 m²
2     63.00 m²
3     51.00 m²
4     42.00 m²
5    500.00 m²
6    245.00 m²
7    100.00 m²
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_descripcion_casa.iloc[:, i] = grupo_descripcion_casa.iloc[:, i].astype(float).apply(formato_monetario_m2)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_11740\3382106320.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0    19,386.00 m²
1    15,794.00 m²
2    14,079.00 m²
3     8,315.00 m²
4     5,440.00 m²
5    10,000.00 m²
6     2,451.00 m²
7    32,240.00 m²
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_descripc

,segmento,Mínimo,Máximo,Promedio
0,A,90.00 m²,"19,386.00 m²",200.19 m²
1,B,70.00 m²,"15,794.00 m²",174.30 m²
2,C,63.00 m²,"14,079.00 m²",150.63 m²
3,D,51.00 m²,"8,315.00 m²",124.03 m²
4,E,42.00 m²,"5,440.00 m²",143.74 m²
5,ELITE,500.00 m²,"10,000.00 m²","1,086.53 m²"
6,L,245.00 m²,"2,451.00 m²",689.32 m²
7,S,100.00 m²,"32,240.00 m²",270.63 m²


In [25]:
# Filtrar los datos según las condiciones especificadas
#filtrado_df = df_total[(~df_total['metros_construido'].isin([0, 1, 2, 3, 4, 5])) & (df_total['precio_m2_construido'] >= 280000)]

# Calcular las estadísticas descriptivas
todo_descripcion = casa_df['metros_total'].describe()[['min', 'max', 'mean']].to_frame().T
todo_descripcion['segmento'] = 'general' # Agregar la columna de segmento
todo_descripcion.rename(columns={
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)

# Calcular el percentil 95%
#percentil_95 = filtrado_df['precio_m2_construido'].quantile(0.95)
# Reemplazar el valor máximo con el percentil 95%
#todo_descripcion['Máximo'] = percentil_95


# Aplicar formato monetario a los valores
def formato_monetario(valor):
    return f"{valor:,.2f} m²"  # Formato monetario con dos decimales

for i in range(3):  # 0, 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
print(todo_descripcion)

                Mínimo        Máximo   Promedio segmento
metros_total  42.00 m²  32,240.00 m²  192.16 m²  general


C:\Users\yoe11\AppData\Local\Temp\ipykernel_11740\1147424691.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'metros_total    42.00 m²
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_11740\1147424691.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'metros_total    32,240.00 m²
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_11740\1147424691.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and w

In [26]:
resultado_final = pd.concat([grupo_descripcion_casa, todo_descripcion], ignore_index=True)
print(resultado_final)

  segmento     Mínimo        Máximo     Promedio
0        A   90.00 m²  19,386.00 m²    200.19 m²
1        B   70.00 m²  15,794.00 m²    174.30 m²
2        C   63.00 m²  14,079.00 m²    150.63 m²
3        D   51.00 m²   8,315.00 m²    124.03 m²
4        E   42.00 m²   5,440.00 m²    143.74 m²
5    ELITE  500.00 m²  10,000.00 m²  1,086.53 m²
6        L  245.00 m²   2,451.00 m²    689.32 m²
7        S  100.00 m²  32,240.00 m²    270.63 m²
8  general   42.00 m²  32,240.00 m²    192.16 m²


In [27]:
orden_segmentos = ['E', 'D', 'C', 'B', 'A', 'S', 'L','ELITE','general']#
ordenado = resultado_final.set_index('segmento').reindex(orden_segmentos).reset_index()
ordenado

,segmento,Mínimo,Máximo,Promedio
0,E,42.00 m²,"5,440.00 m²",143.74 m²
1,D,51.00 m²,"8,315.00 m²",124.03 m²
2,C,63.00 m²,"14,079.00 m²",150.63 m²
3,B,70.00 m²,"15,794.00 m²",174.30 m²
4,A,90.00 m²,"19,386.00 m²",200.19 m²
5,S,100.00 m²,"32,240.00 m²",270.63 m²
6,L,245.00 m²,"2,451.00 m²",689.32 m²
7,ELITE,500.00 m²,"10,000.00 m²","1,086.53 m²"
8,general,42.00 m²,"32,240.00 m²",192.16 m²


In [28]:
# Guardar el JSON en un archivo
resultados_finales_desc_json = resultado_final.to_json(orient='index')
with open('assets/tablas/grupo_terreno_casa.json', 'w') as f:
    f.write(resultados_finales_desc_json)

## M2 Construcción

In [29]:
#casa_df.loc[:, 'segmento'] = casa_df['categoria'].str[0].astype(str)
orden_segmentos = ['E', 'D', 'C', 'B', 'A', 'S', 'L','ELITE']#
grupo_descripcion_casa = casa_df.groupby('segmento')['metros_construido'].describe().reset_index()
grupo_descripcion_casa = grupo_descripcion_casa.loc[:, ['segmento','min', 'max','mean']] #Columnas deseadas
grupo_descripcion_casa.rename(columns={
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)
# Ordenar el dataframe según el orden específico de segmentos
ordenado = grupo_descripcion_casa.set_index('segmento').reindex(orden_segmentos).reset_index()
ordenado

,segmento,Mínimo,Máximo,Promedio
0,E,43.0,9000.0,170.737131
1,D,51.0,472.0,121.628233
2,C,62.0,640.0,162.834052
3,B,80.0,29743.0,234.291722
4,A,80.0,460.0,223.227419
5,S,100.0,1465.0,314.660619
6,L,245.0,1200.0,616.237288
7,ELITE,500.0,6152.0,926.725806


In [30]:
# Función para formatear valores en formato monetario por m2
def formato_monetario_m2(valor):
    return f"{valor:,.2f} m²"  # Formato con $, 2 decimales, separador de miles y unidad m²

# Aplicar formato en m2 a los valores, excluyendo la columna 'segmento'
for i in range(1, 4):  # 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio'
    grupo_descripcion_casa.iloc[:, i] = grupo_descripcion_casa.iloc[:, i].astype(float).apply(formato_monetario_m2)
grupo_descripcion_casa 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_11740\3382106320.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0     80.00 m²
1     80.00 m²
2     62.00 m²
3     51.00 m²
4     43.00 m²
5    500.00 m²
6    245.00 m²
7    100.00 m²
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_descripcion_casa.iloc[:, i] = grupo_descripcion_casa.iloc[:, i].astype(float).apply(formato_monetario_m2)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_11740\3382106320.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       460.00 m²
1    29,743.00 m²
2       640.00 m²
3       472.00 m²
4     9,000.00 m²
5     6,152.00 m²
6     1,200.00 m²
7     1,465.00 m²
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_descripc

,segmento,Mínimo,Máximo,Promedio
0,A,80.00 m²,460.00 m²,223.23 m²
1,B,80.00 m²,"29,743.00 m²",234.29 m²
2,C,62.00 m²,640.00 m²,162.83 m²
3,D,51.00 m²,472.00 m²,121.63 m²
4,E,43.00 m²,"9,000.00 m²",170.74 m²
5,ELITE,500.00 m²,"6,152.00 m²",926.73 m²
6,L,245.00 m²,"1,200.00 m²",616.24 m²
7,S,100.00 m²,"1,465.00 m²",314.66 m²


In [31]:
# Filtrar los datos según las condiciones especificadas
#filtrado_df = df_total[(~df_total['metros_construido'].isin([0, 1, 2, 3, 4, 5])) & (df_total['precio_m2_construido'] >= 280000)]

# Calcular las estadísticas descriptivas
todo_descripcion = casa_df['metros_construido'].describe()[['min', 'max', 'mean']].to_frame().T
todo_descripcion['segmento'] = 'general' # Agregar la columna de segmento
todo_descripcion.rename(columns={
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)

# Calcular el percentil 95%
#percentil_95 = filtrado_df['precio_m2_construido'].quantile(0.95)
# Reemplazar el valor máximo con el percentil 95%
#todo_descripcion['Máximo'] = percentil_95


# Aplicar formato monetario a los valores
def formato_monetario(valor):
    return f"{valor:,.2f} m²"  # Formato monetario con dos decimales

for i in range(3):  # 0, 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
print(todo_descripcion)

                     Mínimo        Máximo   Promedio segmento
metros_construido  43.00 m²  29,743.00 m²  214.09 m²  general


C:\Users\yoe11\AppData\Local\Temp\ipykernel_11740\2239152448.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'metros_construido    43.00 m²
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_11740\2239152448.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'metros_construido    29,743.00 m²
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_11740\2239152448.py:24: FutureWarning: Setting an item of incompatible dtype is deprec

In [32]:
resultado_final = pd.concat([grupo_descripcion_casa, todo_descripcion], ignore_index=True)
print(resultado_final)

  segmento     Mínimo        Máximo   Promedio
0        A   80.00 m²     460.00 m²  223.23 m²
1        B   80.00 m²  29,743.00 m²  234.29 m²
2        C   62.00 m²     640.00 m²  162.83 m²
3        D   51.00 m²     472.00 m²  121.63 m²
4        E   43.00 m²   9,000.00 m²  170.74 m²
5    ELITE  500.00 m²   6,152.00 m²  926.73 m²
6        L  245.00 m²   1,200.00 m²  616.24 m²
7        S  100.00 m²   1,465.00 m²  314.66 m²
8  general   43.00 m²  29,743.00 m²  214.09 m²


In [33]:
orden_segmentos = ['E', 'D', 'C', 'B', 'A', 'S', 'L','ELITE','general']#
ordenado = resultado_final.set_index('segmento').reindex(orden_segmentos).reset_index()
ordenado

,segmento,Mínimo,Máximo,Promedio
0,E,43.00 m²,"9,000.00 m²",170.74 m²
1,D,51.00 m²,472.00 m²,121.63 m²
2,C,62.00 m²,640.00 m²,162.83 m²
3,B,80.00 m²,"29,743.00 m²",234.29 m²
4,A,80.00 m²,460.00 m²,223.23 m²
5,S,100.00 m²,"1,465.00 m²",314.66 m²
6,L,245.00 m²,"1,200.00 m²",616.24 m²
7,ELITE,500.00 m²,"6,152.00 m²",926.73 m²
8,general,43.00 m²,"29,743.00 m²",214.09 m²


In [34]:
# Guardar el JSON en un archivo
resultados_finales_desc_json = resultado_final.to_json(orient='index')
with open('assets/tablas/grupo_construidos_casa.json', 'w') as f:
    f.write(resultados_finales_desc_json)

## Pecio M2 Construcción

In [35]:
casa_df['precio_m2_construido']

0        20432.69
1        10975.61
2        20975.61
3        20975.61
4         1500.00
           ...   
11633     9888.89
11634     8285.71
11635     9250.00
11638    21168.67
11642    11596.64
Name: precio_m2_construido, Length: 10504, dtype: float64

In [36]:
#casa_df.loc[:, 'segmento'] = casa_df['categoria'].str[0].astype(str)
orden_segmentos = ['E', 'D', 'C', 'B', 'A', 'S', 'L','ELITE']#
grupo_descripcion_casa = casa_df.groupby('segmento')['precio_m2_construido'].describe().reset_index()
grupo_descripcion_casa = grupo_descripcion_casa.loc[:, ['segmento','min', 'max','mean']] #Columnas deseadas
grupo_descripcion_casa.rename(columns={
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)
# Ordenar el dataframe según el orden específico de segmentos
ordenado = grupo_descripcion_casa.set_index('segmento').reindex(orden_segmentos).reset_index()
ordenado

,segmento,Mínimo,Máximo,Promedio
0,E,19.28,20541.67,6510.835045
1,D,2400.66,28184.12,12457.393095
2,C,3358.64,32569.92,14037.917484
3,B,107.25,34574.47,15459.360948
4,A,7777.78,47500.00,17206.087210
5,S,2779.52,53703.70,20331.992129
6,L,13333.33,61224.49,28947.668898
7,ELITE,12500.00,91666.67,32629.538387


In [37]:
# Función para formatear valores en formato monetario
def formato_monetario(valor):
    return f"${valor:,.2f}"  # Formato con $ y 2 decimales, separador de miles

# Aplicar formato en m2 a los valores, excluyendo la columna 'segmento'
for i in range(1, 4):  # 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio'
    grupo_descripcion_casa.iloc[:, i] = grupo_descripcion_casa.iloc[:, i].astype(float).apply(formato_monetario)
grupo_descripcion_casa 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_11740\3321038986.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0     $7,777.78
1       $107.25
2     $3,358.64
3     $2,400.66
4        $19.28
5    $12,500.00
6    $13,333.33
7     $2,779.52
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_descripcion_casa.iloc[:, i] = grupo_descripcion_casa.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_11740\3321038986.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0    $47,500.00
1    $34,574.47
2    $32,569.92
3    $28,184.12
4    $20,541.67
5    $91,666.67
6    $61,224.49
7    $53,703.70
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_descripcion_casa.il

,segmento,Mínimo,Máximo,Promedio
0,A,"$7,777.78","$47,500.00","$17,206.09"
1,B,$107.25,"$34,574.47","$15,459.36"
2,C,"$3,358.64","$32,569.92","$14,037.92"
3,D,"$2,400.66","$28,184.12","$12,457.39"
4,E,$19.28,"$20,541.67","$6,510.84"
5,ELITE,"$12,500.00","$91,666.67","$32,629.54"
6,L,"$13,333.33","$61,224.49","$28,947.67"
7,S,"$2,779.52","$53,703.70","$20,331.99"


In [38]:
# Filtrar los datos según las condiciones especificadas
#filtrado_df = df_total[(~df_total['metros_construido'].isin([0, 1, 2, 3, 4, 5])) & (df_total['precio_m2_construido'] >= 280000)]

# Calcular las estadísticas descriptivas
todo_descripcion = casa_df['precio_m2_construido'].describe()[['min', 'max', 'mean']].to_frame().T
todo_descripcion['segmento'] = 'general' # Agregar la columna de segmento
todo_descripcion.rename(columns={
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)

# Calcular el percentil 95%
#percentil_95 = filtrado_df['precio_m2_construido'].quantile(0.95)
# Reemplazar el valor máximo con el percentil 95%
#todo_descripcion['Máximo'] = percentil_95

# Aplicar formato monetario a los valores
def formato_monetario(valor):
    return f"${valor:,.2f}"  # Formato monetario con dos decimales

for i in range(3):  # 0, 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
print(todo_descripcion)

                      Mínimo      Máximo    Promedio segmento
precio_m2_construido  $19.28  $91,666.67  $14,794.98  general


C:\Users\yoe11\AppData\Local\Temp\ipykernel_11740\155160619.py:23: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'precio_m2_construido    $19.28
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_11740\155160619.py:23: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'precio_m2_construido    $91,666.67
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_11740\155160619.py:23: FutureWarning: Setting an item of incompatible dtype is depreca

In [39]:
resultado_final = pd.concat([grupo_descripcion_casa, todo_descripcion], ignore_index=True)
print(resultado_final)

  segmento      Mínimo      Máximo    Promedio
0        A   $7,777.78  $47,500.00  $17,206.09
1        B     $107.25  $34,574.47  $15,459.36
2        C   $3,358.64  $32,569.92  $14,037.92
3        D   $2,400.66  $28,184.12  $12,457.39
4        E      $19.28  $20,541.67   $6,510.84
5    ELITE  $12,500.00  $91,666.67  $32,629.54
6        L  $13,333.33  $61,224.49  $28,947.67
7        S   $2,779.52  $53,703.70  $20,331.99
8  general      $19.28  $91,666.67  $14,794.98


In [40]:
orden_segmentos = ['E', 'D', 'C', 'B', 'A', 'S', 'L','ELITE','general']#
ordenado = resultado_final.set_index('segmento').reindex(orden_segmentos).reset_index()
ordenado

,segmento,Mínimo,Máximo,Promedio
0,E,$19.28,"$20,541.67","$6,510.84"
1,D,"$2,400.66","$28,184.12","$12,457.39"
2,C,"$3,358.64","$32,569.92","$14,037.92"
3,B,$107.25,"$34,574.47","$15,459.36"
4,A,"$7,777.78","$47,500.00","$17,206.09"
5,S,"$2,779.52","$53,703.70","$20,331.99"
6,L,"$13,333.33","$61,224.49","$28,947.67"
7,ELITE,"$12,500.00","$91,666.67","$32,629.54"
8,general,$19.28,"$91,666.67","$14,794.98"


In [41]:
# Guardar el JSON en un archivo
resultados_finales_desc_json = resultado_final.to_json(orient='index')
with open('assets/tablas/grupo_precio_construidos_casa.json', 'w') as f:
    f.write(resultados_finales_desc_json)

# Depas

### Revisión de m2

In [ ]:
# metros_total en casas
precios = depa_df['metros_total'].unique()
precios = np.sort(precios)
precios

array([ 34,  41,  44,  46,  48,  49,  53,  55,  60,  80,  82,  85,  87,
        90,  95,  96, 110, 120, 156])

In [ ]:
# metros_construido en casas
precios = depa_df['metros_construido'].unique()
precios = np.sort(precios)
precios

array([ 34,  41,  44,  46,  48,  49,  53,  55,  60,  75,  80,  82,  85,
        87,  90,  95, 110, 120, 156])

In [ ]:
# metros_total en casas cuando segmento es 'B'
#precios = casa_df[casa_df['segmento'] == 'B']['metros_total'].unique()
precios = depa_df[depa_df['segmento'] == 'B']['metros_total'].value_counts()
#precios = np.sort(precios)
precios

KeyError: 'segmento'

## Depa promedio

In [ ]:
depa_df.loc[:, 'segmento'] = depa_df['categoria'].str[0]
grupo_promedios = round(depa_df.groupby('segmento')[['estacionamiento', 'recamaras', 'baño_total']].mean(),0)
grupo_promedios.astype(int)
# Renombrar las columnas para mayor claridad
grupo_promedios.rename(columns={
    'estacionamiento': 'Estacionamientos',
    'recamaras': 'Recámaras',
    'baño_total': 'Baños',
}, inplace=True)

# Calcular el promedio de estacionamientos, recámaras y baños en general (sin segmento específico)
promedios_generales = round(depa_df[['estacionamiento', 'recamaras', 'baño_total']].mean(),0)
promedios_generales = promedios_generales.astype(int)
# Renombrar las columnas para mayor claridad
promedios_generales.rename(index={
    'estacionamiento': 'Estacionamientos',
    'recamaras': 'Recámaras',
    'baño_total': 'Baños',
}, inplace=True)
# Agregar los promedios generales al dataframe de grupo_promedios
promedios_generales = pd.DataFrame(promedios_generales).T
promedios_generales['segmento'] = 'general'

casa_promedio = pd.concat([grupo_promedios, promedios_generales.set_index('segmento')], ignore_index=False)

#print(grupo_promedios)
#print(promedios_generales)
print(casa_promedio)

# Convertir el dataframe a JSON
grupo_promedios_json = casa_promedio.to_json(orient='index')

# Guardar el JSON en un archivo
with open('assets/tablas/grupo_depa_promedio.json', 'w') as f:
    f.write(grupo_promedios_json)

          Estacionamientos  Recámaras  Baños
segmento                                    
B                      2.0        2.0    2.0
C                      2.0        3.0    3.0
D                      2.0        2.0    1.0
E                      1.0        2.0    1.0
general                1.0        2.0    2.0


## Precio Mercado

In [ ]:
depa_df.loc[:, 'segmento'] = depa_df['categoria'].str[0].astype(str)
orden_segmentos = ['E', 'D', 'C', 'B', ]#'A', 'S']#, 'L'
grupo_descripcion_casa = depa_df.groupby('segmento')['precio'].describe().reset_index()
grupo_descripcion_casa = grupo_descripcion_casa.loc[:, ['segmento','min', 'max','mean']] #Columnas deseadas
grupo_descripcion_casa.rename(columns={
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)
# Ordenar el dataframe según el orden específico de segmentos
ordenado = grupo_descripcion_casa.set_index('segmento').reindex(orden_segmentos).reset_index()
ordenado

,segmento,Mínimo,Máximo,Promedio
0,E,346040.0,980000.0,6.355342e+05
1,D,1249000.0,1712133.0,1.406985e+06
2,C,2250000.0,2500000.0,2.295001e+06
3,B,2650000.0,2850000.0,2.783333e+06


In [ ]:
# Función para formatear valores en formato monetario
def formato_monetario(valor):
    return f"${valor:,.2f}"  # Formato con $ y 2 decimales, separador de miles

# Aplicar formato en m2 a los valores, excluyendo la columna 'segmento'
for i in range(1, 4):  # 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio'
    grupo_descripcion_casa.iloc[:, i] = grupo_descripcion_casa.iloc[:, i].astype(float).apply(formato_monetario)
grupo_descripcion_casa 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_19968\3321038986.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0    $2,650,000.00
1    $2,250,000.00
2    $1,249,000.00
3      $346,040.00
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_descripcion_casa.iloc[:, i] = grupo_descripcion_casa.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_19968\3321038986.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0    $2,850,000.00
1    $2,500,000.00
2    $1,712,133.00
3      $980,000.00
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_descripcion_casa.iloc[:, i] = grupo_descripcion_casa.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppDa

,segmento,Mínimo,Máximo,Promedio
0,B,"$2,650,000.00","$2,850,000.00","$2,783,333.33"
1,C,"$2,250,000.00","$2,500,000.00","$2,295,001.20"
2,D,"$1,249,000.00","$1,712,133.00","$1,406,985.20"
3,E,"$346,040.00","$980,000.00","$635,534.23"


In [ ]:
# Paso 1: Agrupar por segmento y calcular las estadísticas descriptivas
#grupo_descripcion = casa_df.groupby('segmento')['precio'].describe().reset_index()

# Paso 2: Seleccionar las columnas deseadas
#grupo_descripcion = grupo_descripcion[['segmento', 'min', 'max', 'mean']]

# Paso 3: Renombrar las columnas
#grupo_descripcion.rename(columns={
#    'min': 'Mínimo',
#    'max': 'Máximo',
#    'mean': 'Promedio',
#}, inplace=True)

# Paso 4: Calcular el percentil 95% para cada segmento
#percentiles_95 = casa_df.groupby('segmento')['precio'].quantile(0.95).reset_index()
#percentiles_95.rename(columns={'precio': 'Máximo_95%'}, inplace=True)

# Paso 5: Unir los percentiles 95% con el DataFrame original
#grupo_descripcion = grupo_descripcion.merge(percentiles_95, on='segmento')

# Paso 6: Reemplazar el valor máximo con el percentil 95%
#grupo_descripcion['Máximo'] = grupo_descripcion['Máximo_95%']

# Eliminar la columna temporal
#grupo_descripcion.drop(columns=['Máximo_95%'], inplace=True)

# Mostrar el resultado final
#print(grupo_descripcion)

In [ ]:
# Filtrar los datos según las condiciones especificadas
#filtrado_df = df_total[(~df_total['metros_construido'].isin([0, 1, 2, 3, 4, 5])) & (df_total['precio_m2_construido'] >= 280000)]

# Calcular las estadísticas descriptivas
todo_descripcion = depa_df['precio'].describe()[['min', 'max', 'mean']].to_frame().T
todo_descripcion['segmento'] = 'general' # Agregar la columna de segmento
todo_descripcion.rename(columns={
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)

# Calcular el percentil 95%
#percentil_95 = filtrado_df['precio_m2_construido'].quantile(0.95)
# Reemplazar el valor máximo con el percentil 95%
#todo_descripcion['Máximo'] = percentil_95

# Aplicar formato monetario a los valores
def formato_monetario(valor):
    return f"${valor:,.2f}"  # Formato monetario con dos decimales

for i in range(3):  # 0, 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
print(todo_descripcion)

             Mínimo         Máximo       Promedio segmento
precio  $346,040.00  $2,850,000.00  $1,205,026.39  general


C:\Users\yoe11\AppData\Local\Temp\ipykernel_19968\3749309691.py:23: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'precio    $346,040.00
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_19968\3749309691.py:23: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'precio    $2,850,000.00
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_19968\3749309691.py:23: FutureWarning: Setting an item of incompatible dtype is deprecated and will rais

In [ ]:
# Agrupar por segmento y calcular las estadísticas descriptivas
#grupo_descripcion = casa_df.groupby('segmento')['precio'].describe().reset_index()
#grupo_descripcion = grupo_descripcion[['segmento', 'min', 'max', 'mean']] # Seleccionar las columnas deseadas
#grupo_descripcion.rename(columns={'min': 'Mínimo','max': 'Máximo', 'mean': 'Promedio',}, inplace=True)

# Calcular el percentil 95% para cada segmento
#percentiles_95 = df_total.groupby('segmento')['precio'].quantile(0.95).reset_index()
#percentiles_95.rename(columns={'precio': 'Máximo_95%'}, inplace=True)

# Paso 5: Unir los percentiles 95% con el DataFrame original
#grupo_descripcion = grupo_descripcion.merge(percentiles_95, on='segmento')

# Paso 6: Reemplazar el valor máximo con el percentil 95%
#grupo_descripcion['Máximo'] = grupo_descripcion['Máximo_95%']

# Eliminar la columna temporal
#grupo_descripcion.drop(columns=['Máximo_95%'], inplace=True)

# Paso 7: Calcular las estadísticas generales
#general_stats = df_total['precio'].describe()[['min', 'max', 'mean']].to_frame().T
#general_stats.rename(columns={'min': 'Mínimo','max': 'Máximo','mean': 'Promedio',}, inplace=True)

# Calcular el percentil 95% general
#general_percentil_95 = df_total['precio'].quantile(0.95)

# Reemplazar el máximo general con el percentil 95%
#general_stats['Máximo'] = general_percentil_95

# Agregar una fila para los resultados generales
#general_stats['segmento'] = 'General'

# Concatenar los resultados generales al DataFrame de segmentos
#resultado_final = pd.concat([grupo_descripcion, general_stats], ignore_index=True)
#print(resultado_final)

In [ ]:
resultado_final = pd.concat([grupo_descripcion_casa, todo_descripcion], ignore_index=True)
print(resultado_final)

  segmento         Mínimo         Máximo       Promedio
0        B  $2,650,000.00  $2,850,000.00  $2,783,333.33
1        C  $2,250,000.00  $2,500,000.00  $2,295,001.20
2        D  $1,249,000.00  $1,712,133.00  $1,406,985.20
3        E    $346,040.00    $980,000.00    $635,534.23
4  general    $346,040.00  $2,850,000.00  $1,205,026.39


In [ ]:
orden_segmentos = ['E', 'D', 'C', 'B','general']#, 'L' 'A', 'S',
ordenado = resultado_final.set_index('segmento').reindex(orden_segmentos).reset_index()
ordenado

,segmento,Mínimo,Máximo,Promedio
0,E,"$346,040.00","$980,000.00","$635,534.23"
1,D,"$1,249,000.00","$1,712,133.00","$1,406,985.20"
2,C,"$2,250,000.00","$2,500,000.00","$2,295,001.20"
3,B,"$2,650,000.00","$2,850,000.00","$2,783,333.33"
4,general,"$346,040.00","$2,850,000.00","$1,205,026.39"


In [ ]:
# Guardar el JSON en un archivo
resultados_finales_desc_json = resultado_final.to_json(orient='index')
with open('assets/tablas/grupo_precio_mercado_depa.json', 'w') as f:
    f.write(resultados_finales_desc_json)

## M2 Terreno

In [ ]:
depa_df.loc[:, 'segmento'] = depa_df['categoria'].str[0].astype(str)
orden_segmentos = ['E', 'D', 'C', 'B', ]#, 'L''A', 'S'
grupo_descripcion_casa = depa_df.groupby('segmento')['metros_total'].describe().reset_index()
grupo_descripcion_casa = grupo_descripcion_casa.loc[:, ['segmento','min', 'max','mean']] #Columnas deseadas
grupo_descripcion_casa.rename(columns={
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)
# Ordenar el dataframe según el orden específico de segmentos
ordenado = grupo_descripcion_casa.set_index('segmento').reindex(orden_segmentos).reset_index()
ordenado

,segmento,Mínimo,Máximo,Promedio
0,E,34.0,96.0,65.612903
1,D,80.0,120.0,93.400000
2,C,120.0,156.0,148.800000
3,B,95.0,110.0,100.000000


In [ ]:
# Función para formatear valores en formato monetario por m2
def formato_monetario_m2(valor):
    return f"{valor:,.2f} m²"  # Formato con $, 2 decimales, separador de miles y unidad m²

# Aplicar formato en m2 a los valores, excluyendo la columna 'segmento'
for i in range(1, 4):  # 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio'
    grupo_descripcion_casa.iloc[:, i] = grupo_descripcion_casa.iloc[:, i].astype(float).apply(formato_monetario_m2)
grupo_descripcion_casa 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_19968\3382106320.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0     95.00 m²
1    120.00 m²
2     80.00 m²
3     34.00 m²
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_descripcion_casa.iloc[:, i] = grupo_descripcion_casa.iloc[:, i].astype(float).apply(formato_monetario_m2)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_19968\3382106320.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0    110.00 m²
1    156.00 m²
2    120.00 m²
3     96.00 m²
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_descripcion_casa.iloc[:, i] = grupo_descripcion_casa.iloc[:, i].astype(float).apply(formato_monetario_m2)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_19

,segmento,Mínimo,Máximo,Promedio
0,B,95.00 m²,110.00 m²,100.00 m²
1,C,120.00 m²,156.00 m²,148.80 m²
2,D,80.00 m²,120.00 m²,93.40 m²
3,E,34.00 m²,96.00 m²,65.61 m²


In [ ]:
# Paso 1: Agrupar por segmento y calcular las estadísticas descriptivas
#grupo_descripcion = casa_df.groupby('segmento')['precio'].describe().reset_index()

# Paso 2: Seleccionar las columnas deseadas
#grupo_descripcion = grupo_descripcion[['segmento', 'min', 'max', 'mean']]

# Paso 3: Renombrar las columnas
#grupo_descripcion.rename(columns={
#    'min': 'Mínimo',
#    'max': 'Máximo',
#    'mean': 'Promedio',
#}, inplace=True)

# Paso 4: Calcular el percentil 95% para cada segmento
#percentiles_95 = casa_df.groupby('segmento')['precio'].quantile(0.95).reset_index()
#percentiles_95.rename(columns={'precio': 'Máximo_95%'}, inplace=True)

# Paso 5: Unir los percentiles 95% con el DataFrame original
#grupo_descripcion = grupo_descripcion.merge(percentiles_95, on='segmento')

# Paso 6: Reemplazar el valor máximo con el percentil 95%
#grupo_descripcion['Máximo'] = grupo_descripcion['Máximo_95%']

# Eliminar la columna temporal
#grupo_descripcion.drop(columns=['Máximo_95%'], inplace=True)

# Mostrar el resultado final
#print(grupo_descripcion)

In [ ]:
# Filtrar los datos según las condiciones especificadas
#filtrado_df = df_total[(~df_total['metros_construido'].isin([0, 1, 2, 3, 4, 5])) & (df_total['precio_m2_construido'] >= 280000)]

# Calcular las estadísticas descriptivas
todo_descripcion = depa_df['metros_total'].describe()[['min', 'max', 'mean']].to_frame().T
todo_descripcion['segmento'] = 'general' # Agregar la columna de segmento
todo_descripcion.rename(columns={
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)

# Calcular el percentil 95%
#percentil_95 = filtrado_df['precio_m2_construido'].quantile(0.95)
# Reemplazar el valor máximo con el percentil 95%
#todo_descripcion['Máximo'] = percentil_95


# Aplicar formato monetario a los valores
def formato_monetario(valor):
    return f"{valor:,.2f} m²"  # Formato monetario con dos decimales

for i in range(3):  # 0, 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
print(todo_descripcion)

                Mínimo     Máximo  Promedio segmento
metros_total  34.00 m²  156.00 m²  88.07 m²  general


C:\Users\yoe11\AppData\Local\Temp\ipykernel_19968\2326618923.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'metros_total    34.00 m²
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_19968\2326618923.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'metros_total    156.00 m²
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_19968\2326618923.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will

In [ ]:
# Agrupar por segmento y calcular las estadísticas descriptivas
#grupo_descripcion = casa_df.groupby('segmento')['precio'].describe().reset_index()
#grupo_descripcion = grupo_descripcion[['segmento', 'min', 'max', 'mean']] # Seleccionar las columnas deseadas
#grupo_descripcion.rename(columns={'min': 'Mínimo','max': 'Máximo', 'mean': 'Promedio',}, inplace=True)

# Calcular el percentil 95% para cada segmento
#percentiles_95 = df_total.groupby('segmento')['precio'].quantile(0.95).reset_index()
#percentiles_95.rename(columns={'precio': 'Máximo_95%'}, inplace=True)

# Paso 5: Unir los percentiles 95% con el DataFrame original
#grupo_descripcion = grupo_descripcion.merge(percentiles_95, on='segmento')

# Paso 6: Reemplazar el valor máximo con el percentil 95%
#grupo_descripcion['Máximo'] = grupo_descripcion['Máximo_95%']

# Eliminar la columna temporal
#grupo_descripcion.drop(columns=['Máximo_95%'], inplace=True)

# Paso 7: Calcular las estadísticas generales
#general_stats = df_total['precio'].describe()[['min', 'max', 'mean']].to_frame().T
#general_stats.rename(columns={'min': 'Mínimo','max': 'Máximo','mean': 'Promedio',}, inplace=True)

# Calcular el percentil 95% general
#general_percentil_95 = df_total['precio'].quantile(0.95)

# Reemplazar el máximo general con el percentil 95%
#general_stats['Máximo'] = general_percentil_95

# Agregar una fila para los resultados generales
#general_stats['segmento'] = 'General'

# Concatenar los resultados generales al DataFrame de segmentos
#resultado_final = pd.concat([grupo_descripcion, general_stats], ignore_index=True)
#print(resultado_final)

In [ ]:
resultado_final = pd.concat([grupo_descripcion_casa, todo_descripcion], ignore_index=True)
print(resultado_final)

  segmento     Mínimo     Máximo   Promedio
0        B   95.00 m²  110.00 m²  100.00 m²
1        C  120.00 m²  156.00 m²  148.80 m²
2        D   80.00 m²  120.00 m²   93.40 m²
3        E   34.00 m²   96.00 m²   65.61 m²
4  general   34.00 m²  156.00 m²   88.07 m²


In [ ]:
orden_segmentos = ['E', 'D', 'C', 'B', 'general']#, 'L''A', 'S',
ordenado = resultado_final.set_index('segmento').reindex(orden_segmentos).reset_index()
ordenado

,segmento,Mínimo,Máximo,Promedio
0,E,34.00 m²,96.00 m²,65.61 m²
1,D,80.00 m²,120.00 m²,93.40 m²
2,C,120.00 m²,156.00 m²,148.80 m²
3,B,95.00 m²,110.00 m²,100.00 m²
4,general,34.00 m²,156.00 m²,88.07 m²


In [ ]:
# Guardar el JSON en un archivo
resultados_finales_desc_json = resultado_final.to_json(orient='index')
with open('assets/tablas/grupo_terreno_depa.json', 'w') as f:
    f.write(resultados_finales_desc_json)

## M2 Construcción

In [ ]:
depa_df.loc[:, 'segmento'] = depa_df['categoria'].str[0].astype(str)
orden_segmentos = ['E', 'D', 'C', 'B',]#, 'L' 'A', 'S'
grupo_descripcion_casa = depa_df.groupby('segmento')['metros_construido'].describe().reset_index()
grupo_descripcion_casa = grupo_descripcion_casa.loc[:, ['segmento','min', 'max','mean']] #Columnas deseadas
grupo_descripcion_casa.rename(columns={
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)
# Ordenar el dataframe según el orden específico de segmentos
ordenado = grupo_descripcion_casa.set_index('segmento').reindex(orden_segmentos).reset_index()
ordenado

,segmento,Mínimo,Máximo,Promedio
0,E,34.0,90.0,64.935484
1,D,80.0,120.0,93.400000
2,C,120.0,156.0,148.800000
3,B,95.0,110.0,100.000000


In [ ]:
# Función para formatear valores en formato monetario por m2
def formato_monetario_m2(valor):
    return f"{valor:,.2f} m²"  # Formato con $, 2 decimales, separador de miles y unidad m²

# Aplicar formato en m2 a los valores, excluyendo la columna 'segmento'
for i in range(1, 4):  # 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio'
    grupo_descripcion_casa.iloc[:, i] = grupo_descripcion_casa.iloc[:, i].astype(float).apply(formato_monetario_m2)
grupo_descripcion_casa 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_19968\3382106320.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0     95.00 m²
1    120.00 m²
2     80.00 m²
3     34.00 m²
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_descripcion_casa.iloc[:, i] = grupo_descripcion_casa.iloc[:, i].astype(float).apply(formato_monetario_m2)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_19968\3382106320.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0    110.00 m²
1    156.00 m²
2    120.00 m²
3     90.00 m²
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_descripcion_casa.iloc[:, i] = grupo_descripcion_casa.iloc[:, i].astype(float).apply(formato_monetario_m2)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_19

,segmento,Mínimo,Máximo,Promedio
0,B,95.00 m²,110.00 m²,100.00 m²
1,C,120.00 m²,156.00 m²,148.80 m²
2,D,80.00 m²,120.00 m²,93.40 m²
3,E,34.00 m²,90.00 m²,64.94 m²


In [ ]:
# Paso 1: Agrupar por segmento y calcular las estadísticas descriptivas
#grupo_descripcion = casa_df.groupby('segmento')['precio'].describe().reset_index()

# Paso 2: Seleccionar las columnas deseadas
#grupo_descripcion = grupo_descripcion[['segmento', 'min', 'max', 'mean']]

# Paso 3: Renombrar las columnas
#grupo_descripcion.rename(columns={
#    'min': 'Mínimo',
#    'max': 'Máximo',
#    'mean': 'Promedio',
#}, inplace=True)

# Paso 4: Calcular el percentil 95% para cada segmento
#percentiles_95 = casa_df.groupby('segmento')['precio'].quantile(0.95).reset_index()
#percentiles_95.rename(columns={'precio': 'Máximo_95%'}, inplace=True)

# Paso 5: Unir los percentiles 95% con el DataFrame original
#grupo_descripcion = grupo_descripcion.merge(percentiles_95, on='segmento')

# Paso 6: Reemplazar el valor máximo con el percentil 95%
#grupo_descripcion['Máximo'] = grupo_descripcion['Máximo_95%']

# Eliminar la columna temporal
#grupo_descripcion.drop(columns=['Máximo_95%'], inplace=True)

# Mostrar el resultado final
#print(grupo_descripcion)

In [ ]:
# Filtrar los datos según las condiciones especificadas
#filtrado_df = df_total[(~df_total['metros_construido'].isin([0, 1, 2, 3, 4, 5])) & (df_total['precio_m2_construido'] >= 280000)]

# Calcular las estadísticas descriptivas
todo_descripcion = depa_df['metros_construido'].describe()[['min', 'max', 'mean']].to_frame().T
todo_descripcion['segmento'] = 'general' # Agregar la columna de segmento
todo_descripcion.rename(columns={
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)

# Calcular el percentil 95%
#percentil_95 = filtrado_df['precio_m2_construido'].quantile(0.95)
# Reemplazar el valor máximo con el percentil 95%
#todo_descripcion['Máximo'] = percentil_95


# Aplicar formato monetario a los valores
def formato_monetario(valor):
    return f"{valor:,.2f} m²"  # Formato monetario con dos decimales

for i in range(3):  # 0, 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
print(todo_descripcion)

                     Mínimo     Máximo  Promedio segmento
metros_construido  34.00 m²  156.00 m²  87.69 m²  general


C:\Users\yoe11\AppData\Local\Temp\ipykernel_19968\4103975992.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'metros_construido    34.00 m²
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_19968\4103975992.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'metros_construido    156.00 m²
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_19968\4103975992.py:24: FutureWarning: Setting an item of incompatible dtype is deprecate

In [ ]:
# Agrupar por segmento y calcular las estadísticas descriptivas
#grupo_descripcion = casa_df.groupby('segmento')['precio'].describe().reset_index()
#grupo_descripcion = grupo_descripcion[['segmento', 'min', 'max', 'mean']] # Seleccionar las columnas deseadas
#grupo_descripcion.rename(columns={'min': 'Mínimo','max': 'Máximo', 'mean': 'Promedio',}, inplace=True)

# Calcular el percentil 95% para cada segmento
#percentiles_95 = df_total.groupby('segmento')['precio'].quantile(0.95).reset_index()
#percentiles_95.rename(columns={'precio': 'Máximo_95%'}, inplace=True)

# Paso 5: Unir los percentiles 95% con el DataFrame original
#grupo_descripcion = grupo_descripcion.merge(percentiles_95, on='segmento')

# Paso 6: Reemplazar el valor máximo con el percentil 95%
#grupo_descripcion['Máximo'] = grupo_descripcion['Máximo_95%']

# Eliminar la columna temporal
#grupo_descripcion.drop(columns=['Máximo_95%'], inplace=True)

# Paso 7: Calcular las estadísticas generales
#general_stats = df_total['precio'].describe()[['min', 'max', 'mean']].to_frame().T
#general_stats.rename(columns={'min': 'Mínimo','max': 'Máximo','mean': 'Promedio',}, inplace=True)

# Calcular el percentil 95% general
#general_percentil_95 = df_total['precio'].quantile(0.95)

# Reemplazar el máximo general con el percentil 95%
#general_stats['Máximo'] = general_percentil_95

# Agregar una fila para los resultados generales
#general_stats['segmento'] = 'General'

# Concatenar los resultados generales al DataFrame de segmentos
#resultado_final = pd.concat([grupo_descripcion, general_stats], ignore_index=True)
#print(resultado_final)

In [ ]:
resultado_final = pd.concat([grupo_descripcion_casa, todo_descripcion], ignore_index=True)
print(resultado_final)

  segmento     Mínimo     Máximo   Promedio
0        B   95.00 m²  110.00 m²  100.00 m²
1        C  120.00 m²  156.00 m²  148.80 m²
2        D   80.00 m²  120.00 m²   93.40 m²
3        E   34.00 m²   90.00 m²   64.94 m²
4  general   34.00 m²  156.00 m²   87.69 m²


In [ ]:
orden_segmentos = ['E', 'D', 'C', 'B', 'general']#, 'L''A', 'S',
ordenado = resultado_final.set_index('segmento').reindex(orden_segmentos).reset_index()
ordenado

,segmento,Mínimo,Máximo,Promedio
0,E,34.00 m²,90.00 m²,64.94 m²
1,D,80.00 m²,120.00 m²,93.40 m²
2,C,120.00 m²,156.00 m²,148.80 m²
3,B,95.00 m²,110.00 m²,100.00 m²
4,general,34.00 m²,156.00 m²,87.69 m²


In [ ]:
# Guardar el JSON en un archivo
resultados_finales_desc_json = resultado_final.to_json(orient='index')
with open('assets/tablas/grupo_construidos_depa.json', 'w') as f:
    f.write(resultados_finales_desc_json)

## Pecio M2 Construcción

In [ ]:
depa_df['precio_m2_construido']

16     30000.00
22     20416.67
24     14423.08
33     20879.67
37     19047.41
42     10833.33
44     13888.89
46     13147.37
62     11951.22
64     10666.67
65     11294.12
77     13373.17
82      9146.34
90      7730.31
95     10333.33
100    10909.09
121    11304.35
127     8750.00
128    10243.90
131     3977.47
231    14673.91
234    11166.67
258     3977.47
259     9716.98
260    11304.35
263    10333.33
264    10333.33
265     7730.31
266     9146.34
268    11294.12
269    11951.22
270    13147.37
271    13888.89
272    10833.33
273    19047.41
274    20833.33
275    30000.00
277    11764.71
278    10227.27
279    10194.90
280     9583.33
290    20879.67
326    24090.91
354    14423.08
355    14423.08
356    14423.08
357    14423.08
358    14423.08
359    14423.10
360    14423.13
469    11166.67
588    10333.33
707    10333.33
806     6499.56
Name: precio_m2_construido, dtype: float64

In [ ]:
depa_df.loc[:, 'segmento'] = depa_df['categoria'].str[0].astype(str)
orden_segmentos = ['E', 'D', 'C', 'B',]#, 'L' 'A', 'S'
grupo_descripcion_casa = depa_df.groupby('segmento')['precio_m2_construido'].describe().reset_index()
grupo_descripcion_casa = grupo_descripcion_casa.loc[:, ['segmento','min', 'max','mean']] #Columnas deseadas
grupo_descripcion_casa.rename(columns={
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)
# Ordenar el dataframe según el orden específico de segmentos
ordenado = grupo_descripcion_casa.set_index('segmento').reindex(orden_segmentos).reset_index()
ordenado

,segmento,Mínimo,Máximo,Promedio
0,E,3977.47,14673.91,10045.519355
1,D,10833.33,20879.67,15559.334000
2,C,14423.08,20833.33,15663.471000
3,B,24090.91,30000.00,28030.303333


In [ ]:
# Función para formatear valores en formato monetario
def formato_monetario(valor):
    return f"${valor:,.2f}"  # Formato con $ y 2 decimales, separador de miles

# Aplicar formato en m2 a los valores, excluyendo la columna 'segmento'
for i in range(1, 4):  # 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio'
    grupo_descripcion_casa.iloc[:, i] = grupo_descripcion_casa.iloc[:, i].astype(float).apply(formato_monetario)
grupo_descripcion_casa 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_19968\3321038986.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0    $24,090.91
1    $14,423.08
2    $10,833.33
3     $3,977.47
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_descripcion_casa.iloc[:, i] = grupo_descripcion_casa.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_19968\3321038986.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0    $30,000.00
1    $20,833.33
2    $20,879.67
3    $14,673.91
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_descripcion_casa.iloc[:, i] = grupo_descripcion_casa.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_

,segmento,Mínimo,Máximo,Promedio
0,B,"$24,090.91","$30,000.00","$28,030.30"
1,C,"$14,423.08","$20,833.33","$15,663.47"
2,D,"$10,833.33","$20,879.67","$15,559.33"
3,E,"$3,977.47","$14,673.91","$10,045.52"


In [ ]:
# Paso 1: Agrupar por segmento y calcular las estadísticas descriptivas
#grupo_descripcion = casa_df.groupby('segmento')['precio'].describe().reset_index()

# Paso 2: Seleccionar las columnas deseadas
#grupo_descripcion = grupo_descripcion[['segmento', 'min', 'max', 'mean']]

# Paso 3: Renombrar las columnas
#grupo_descripcion.rename(columns={
#    'min': 'Mínimo',
#    'max': 'Máximo',
#    'mean': 'Promedio',
#}, inplace=True)

# Paso 4: Calcular el percentil 95% para cada segmento
#percentiles_95 = casa_df.groupby('segmento')['precio'].quantile(0.95).reset_index()
#percentiles_95.rename(columns={'precio': 'Máximo_95%'}, inplace=True)

# Paso 5: Unir los percentiles 95% con el DataFrame original
#grupo_descripcion = grupo_descripcion.merge(percentiles_95, on='segmento')

# Paso 6: Reemplazar el valor máximo con el percentil 95%
#grupo_descripcion['Máximo'] = grupo_descripcion['Máximo_95%']

# Eliminar la columna temporal
#grupo_descripcion.drop(columns=['Máximo_95%'], inplace=True)

# Mostrar el resultado final
#print(grupo_descripcion)

In [ ]:
# Filtrar los datos según las condiciones especificadas
#filtrado_df = df_total[(~df_total['metros_construido'].isin([0, 1, 2, 3, 4, 5])) & (df_total['precio_m2_construido'] >= 280000)]

# Calcular las estadísticas descriptivas
todo_descripcion = depa_df['precio_m2_construido'].describe()[['min', 'max', 'mean']].to_frame().T
todo_descripcion['segmento'] = 'general' # Agregar la columna de segmento
todo_descripcion.rename(columns={
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)

# Calcular el percentil 95%
#percentil_95 = filtrado_df['precio_m2_construido'].quantile(0.95)
# Reemplazar el valor máximo con el percentil 95%
#todo_descripcion['Máximo'] = percentil_95

# Aplicar formato monetario a los valores
def formato_monetario(valor):
    return f"${valor:,.2f}"  # Formato monetario con dos decimales

for i in range(3):  # 0, 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
print(todo_descripcion)

                         Mínimo      Máximo    Promedio segmento
precio_m2_construido  $3,977.47  $30,000.00  $13,106.11  general


C:\Users\yoe11\AppData\Local\Temp\ipykernel_19968\1082635710.py:23: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'precio_m2_construido    $3,977.47
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_19968\1082635710.py:23: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'precio_m2_construido    $30,000.00
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_19968\1082635710.py:23: FutureWarning: Setting an item of incompatible dtype is d

In [ ]:
# Agrupar por segmento y calcular las estadísticas descriptivas
#grupo_descripcion = casa_df.groupby('segmento')['precio'].describe().reset_index()
#grupo_descripcion = grupo_descripcion[['segmento', 'min', 'max', 'mean']] # Seleccionar las columnas deseadas
#grupo_descripcion.rename(columns={'min': 'Mínimo','max': 'Máximo', 'mean': 'Promedio',}, inplace=True)

# Calcular el percentil 95% para cada segmento
#percentiles_95 = df_total.groupby('segmento')['precio'].quantile(0.95).reset_index()
#percentiles_95.rename(columns={'precio': 'Máximo_95%'}, inplace=True)

# Paso 5: Unir los percentiles 95% con el DataFrame original
#grupo_descripcion = grupo_descripcion.merge(percentiles_95, on='segmento')

# Paso 6: Reemplazar el valor máximo con el percentil 95%
#grupo_descripcion['Máximo'] = grupo_descripcion['Máximo_95%']

# Eliminar la columna temporal
#grupo_descripcion.drop(columns=['Máximo_95%'], inplace=True)

# Paso 7: Calcular las estadísticas generales
#general_stats = df_total['precio'].describe()[['min', 'max', 'mean']].to_frame().T
#general_stats.rename(columns={'min': 'Mínimo','max': 'Máximo','mean': 'Promedio',}, inplace=True)

# Calcular el percentil 95% general
#general_percentil_95 = df_total['precio'].quantile(0.95)

# Reemplazar el máximo general con el percentil 95%
#general_stats['Máximo'] = general_percentil_95

# Agregar una fila para los resultados generales
#general_stats['segmento'] = 'General'

# Concatenar los resultados generales al DataFrame de segmentos
#resultado_final = pd.concat([grupo_descripcion, general_stats], ignore_index=True)
#print(resultado_final)

In [ ]:
resultado_final = pd.concat([grupo_descripcion_casa, todo_descripcion], ignore_index=True)
print(resultado_final)

  segmento      Mínimo      Máximo    Promedio
0        B  $24,090.91  $30,000.00  $28,030.30
1        C  $14,423.08  $20,833.33  $15,663.47
2        D  $10,833.33  $20,879.67  $15,559.33
3        E   $3,977.47  $14,673.91  $10,045.52
4  general   $3,977.47  $30,000.00  $13,106.11


In [ ]:
orden_segmentos = ['E', 'D', 'C', 'B','general']#, 'L', 'A', 'S'
ordenado = resultado_final.set_index('segmento').reindex(orden_segmentos).reset_index()
ordenado

,segmento,Mínimo,Máximo,Promedio
0,E,"$3,977.47","$14,673.91","$10,045.52"
1,D,"$10,833.33","$20,879.67","$15,559.33"
2,C,"$14,423.08","$20,833.33","$15,663.47"
3,B,"$24,090.91","$30,000.00","$28,030.30"
4,general,"$3,977.47","$30,000.00","$13,106.11"


In [ ]:
# Guardar el JSON en un archivo
resultados_finales_desc_json = resultado_final.to_json(orient='index')
with open('assets/tablas/grupo_precio_construidos_depa.json', 'w') as f:
    f.write(resultados_finales_desc_json)

# _____
# ANTES

In [7]:
rangos_precio = {
    "E1": (0, 500000),
    "E2": (500001, 750000),
    "E3": (750001, 1000000),
    "D1": (1000001, 1250000),
    "D2": (1250001, 1500000),
    "D3": (1500001, 1750000),
    "C1": (1750001, 2000000),
    "C2": (2000001, 2250000),
    "C3": (2250001, 2500000),
    "B1": (2500001, 2750000),
    "B2": (2750001, 3000000),
    "B3": (3000001, 3250000),
    "A1": (3250001, 3500000),
    "A2": (3500001, 3750000),
    "A3": (3750001, 4000000),
    "S1": (4000001, 6000000),
    "S2": (6000001, 8000000),
    "S3": (8000001, 12000000),
    "L1": (12000001, 14000000),
    "L2": (14000001, 16000000),
    "L3": (16000001, 18000000),
    "L+": (18000001, 22000000),
    "ELITE": (22000001, float('inf')),
}

# Función para asignar la categoría según el precio
#def asignar_categoria(precio):
#    for categoria, (limite_inferior, limite_superior) in rangos_precio.items():
#        if limite_inferior <= precio < limite_superior:
#            return categoria
#    return None  # En caso de que el precio no caiga en ningún rango (caso raro)
def asignar_categoria(precio):
    for categoria, (limite_inferior, limite_superior) in rangos_precio.items():
        if limite_inferior <= precio < limite_superior:
            return categoria
    return "Sin categoría"  # Asignar una etiqueta clara en lugar de None
df_total = df_total.drop(columns=['categoria'], errors='ignore')


# Convertir 'precio' a numérico y manejar errores para tener la categoría correcta 
df_total['precio'] = pd.to_numeric(df_total['precio'], errors='coerce')
# Llenar valores faltantes con un número por defecto, por ejemplo, 0
df_total['precio'] = df_total['precio'].fillna(0).astype(int)
df_total = df_total[df_total['precio'] > 0]


#df_total['categoria'] = df_total['precio'].apply(asignar_categoria)
df_total['categoria'] = df_total['precio'].apply(asignar_categoria)
categorias_invalidas = df_total[df_total['categoria'].isnull()]
if not categorias_invalidas.empty:
    print("Precios sin categoría:")
    print(categorias_invalidas[['precio']])


In [8]:
import numpy as np 
condition = (df_total['baño_total'] % 1 == 0.5) # Valores que contienen .5    
# Convertir al número entero anterior
df_total.loc[condition, 'baño_total'] = np.floor(df_total['baño_total']) 
df_total['baño_total'] = df_total['baño_total'].clip(lower=1,upper=8) # Limitar los valores 
df_total['estacionamiento'] = df_total['estacionamiento'].clip(lower=1,)#Mínimo 1 #upper=8
df_total['recamaras'] = df_total['recamaras'].clip(lower=1,)#upper=8 

df_total = df_total[df_total['precio'] != 0]
df_total = df_total[df_total['precio'] > 4800]
df_total = df_total[df_total['metros_total'] > 20]
df_total = df_total[df_total['metros_construido'] > 20]

- Casa promedio
- Casa Mediana
- Precio mercado

- m2 Terreno
- M2 Construcción
- Precio m2 construcción

In [9]:
casas_df = df_total[df_total['Tipo'].isin(['Casa', 'Casa En Fraccionamiento', 'Casa En Condominio', 'Casa Duplex'])]
departamentos_df = df_total[df_total['Tipo'].isin(['Departamento','Duplex','Condominio Horizontal'])]

## Casa promedio

In [10]:
casas_df.loc[:, 'segmento'] = casas_df['categoria'].str[0]
grupo_promedios = round(casas_df.groupby('segmento')[['estacionamiento', 'recamaras', 'baño_total']].mean(),0)
grupo_promedios.astype(int)
# Renombrar las columnas para mayor claridad
grupo_promedios.rename(columns={
    'estacionamiento': 'Estacionamientos',
    'recamaras': 'Recámaras',
    'baño_total': 'Baños',
}, inplace=True)

# Calcular el promedio de estacionamientos, recámaras y baños en general (sin segmento específico)
promedios_generales = round(casas_df[['estacionamiento', 'recamaras', 'baño_total']].mean(),0)
promedios_generales = promedios_generales.astype(int)
# Renombrar las columnas para mayor claridad
promedios_generales.rename(index={
    'estacionamiento': 'Estacionamientos',
    'recamaras': 'Recámaras',
    'baño_total': 'Baños',
}, inplace=True)
# Agregar los promedios generales al dataframe de grupo_promedios
promedios_generales = pd.DataFrame(promedios_generales).T
promedios_generales['segmento'] = 'general'

casa_promedio = pd.concat([grupo_promedios, promedios_generales.set_index('segmento')], ignore_index=False)

print(grupo_promedios)
print(promedios_generales)
print(casa_promedio)

# Convertir el dataframe a JSON
grupo_promedios_json = casa_promedio.to_json(orient='index')

# Guardar el JSON en un archivo
with open('assets/tablas/grupo_casa_promedio.json', 'w') as f:
    f.write(grupo_promedios_json)

          Estacionamientos  Recámaras  Baños
segmento                                    
A                      2.0        4.0    2.0
B                      2.0        3.0    2.0
C                      2.0        3.0    2.0
D                      2.0        3.0    2.0
E                      2.0        3.0    2.0
L                      2.0        4.0    3.0
S                      2.0        4.0    2.0
   Estacionamientos  Recámaras  Baños segmento
0                 2          3      2  general
          Estacionamientos  Recámaras  Baños
segmento                                    
A                      2.0        4.0    2.0
B                      2.0        3.0    2.0
C                      2.0        3.0    2.0
D                      2.0        3.0    2.0
E                      2.0        3.0    2.0
L                      2.0        4.0    3.0
S                      2.0        4.0    2.0
general                2.0        3.0    2.0


C:\Users\yoe11\AppData\Local\Temp\ipykernel_2772\2149301391.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  casas_df.loc[:, 'segmento'] = casas_df['categoria'].str[0]


## Depa Promedio

In [11]:
departamentos_df.loc[:, 'segmento'] = departamentos_df['categoria'].str[0]
grupo_promedios = round(departamentos_df.groupby('segmento')[['estacionamiento', 'recamaras', 'baño_total']].mean(),0)
grupo_promedios.astype(int)
# Renombrar las columnas para mayor claridad
grupo_promedios.rename(columns={
    'estacionamiento': 'Estacionamientos',
    'recamaras': 'Recámaras',
    'baño_total': 'Baños',
}, inplace=True)

# Calcular el promedio de estacionamientos, recámaras y baños en general (sin segmento específico)
promedios_generales = round(departamentos_df[['estacionamiento', 'recamaras', 'baño_total']].mean(),0)
promedios_generales = promedios_generales.astype(int)
# Renombrar las columnas para mayor claridad
promedios_generales.rename(index={
    'estacionamiento': 'Estacionamientos',
    'recamaras': 'Recámaras',
    'baño_total': 'Baños',
}, inplace=True)
# Agregar los promedios generales al dataframe de grupo_promedios
promedios_generales = pd.DataFrame(promedios_generales).T
promedios_generales['segmento'] = 'general'

casa_promedio = pd.concat([grupo_promedios, promedios_generales.set_index('segmento')], ignore_index=False)

print(grupo_promedios)
print(promedios_generales)
print(casa_promedio)

# Convertir el dataframe a JSON
grupo_promedios_json = casa_promedio.to_json(orient='index')

# Guardar el JSON en un archivo
with open('assets/tablas/grupo_depa_promedio.json', 'w') as f:
    f.write(grupo_promedios_json)

          Estacionamientos  Recámaras  Baños
segmento                                    
A                      1.0        3.0    2.0
B                      1.0        2.0    2.0
C                      1.0        2.0    1.0
D                      1.0        2.0    1.0
E                      1.0        2.0    1.0
L                      2.0        3.0    1.0
S                      2.0        3.0    2.0
   Estacionamientos  Recámaras  Baños segmento
0                 1          2      2  general
          Estacionamientos  Recámaras  Baños
segmento                                    
A                      1.0        3.0    2.0
B                      1.0        2.0    2.0
C                      1.0        2.0    1.0
D                      1.0        2.0    1.0
E                      1.0        2.0    1.0
L                      2.0        3.0    1.0
S                      2.0        3.0    2.0
general                1.0        2.0    2.0


C:\Users\yoe11\AppData\Local\Temp\ipykernel_2772\3929789102.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  departamentos_df.loc[:, 'segmento'] = departamentos_df['categoria'].str[0]


## Precio Mercado

In [12]:
df_total['segmento'] = df_total['categoria'].str[0]
grupo_descripcion = df_total.groupby('segmento')['precio'].describe().reset_index()
grupo_descripcion = grupo_descripcion[['segmento','min', 'max','mean']] #Columnas deseadas
grupo_descripcion.rename(columns={
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)
grupo_descripcion 

,segmento,Mínimo,Máximo,Promedio
0,A,3255000.0,3999000.0,3.601984e+06
1,B,2500002.0,3242320.0,2.881962e+06
2,C,1750050.0,2499000.0,2.115324e+06
3,D,1005000.0,1749605.0,1.396118e+06
4,E,55000.0,90000000.0,1.747006e+06
5,L,12100000.0,21000000.0,1.618773e+07
6,S,500000.0,22000000.0,5.003589e+06


In [13]:
grupo_descripcion = df_total.groupby('segmento')['precio'].describe().reset_index()
grupo_descripcion = grupo_descripcion[['segmento', 'min', 'max', 'mean']]
grupo_descripcion.rename(columns={
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)

# Calcular el percentil 95% para cada segmento
percentiles_95 = df_total.groupby('segmento')['precio'].quantile(0.95).reset_index()
percentiles_95.rename(columns={'precio': 'Máximo_95%'}, inplace=True)
grupo_descripcion = grupo_descripcion.merge(percentiles_95, on='segmento')
# Reemplazar el valor máximo con el percentil 95%
grupo_descripcion['Máximo'] = grupo_descripcion['Máximo_95%']

# Eliminar la columna temporal
grupo_descripcion.drop(columns=['Máximo_95%'], inplace=True)
print(grupo_descripcion)


  segmento      Mínimo      Máximo      Promedio
0        A   3255000.0   3950000.0  3.601984e+06
1        B   2500002.0   3200000.0  2.881962e+06
2        C   1750050.0   2450000.0  2.115324e+06
3        D   1005000.0   1700000.0  1.396118e+06
4        E     55000.0    989900.0  1.747006e+06
5        L  12100000.0  21000000.0  1.618773e+07
6        S    500000.0   9985000.0  5.003589e+06


In [14]:
# Función para formatear valores en formato monetario
def formato_monetario(valor):
    return f"${valor:,.2f}"  # Formato con $ y 2 decimales, separador de miles

# Aplicar formato en m2 a los valores, excluyendo la columna 'segmento'
for i in range(1, 4):  # 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio'
    grupo_descripcion.iloc[:, i] = grupo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
grupo_descripcion 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_2772\1329763181.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0     $3,255,000.00
1     $2,500,002.00
2     $1,750,050.00
3     $1,005,000.00
4        $55,000.00
5    $12,100,000.00
6       $500,000.00
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_descripcion.iloc[:, i] = grupo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_2772\1329763181.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0     $3,950,000.00
1     $3,200,000.00
2     $2,450,000.00
3     $1,700,000.00
4       $989,900.00
5    $21,000,000.00
6     $9,985,000.00
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_descrip

,segmento,Mínimo,Máximo,Promedio
0,A,"$3,255,000.00","$3,950,000.00","$3,601,983.91"
1,B,"$2,500,002.00","$3,200,000.00","$2,881,961.62"
2,C,"$1,750,050.00","$2,450,000.00","$2,115,324.05"
3,D,"$1,005,000.00","$1,700,000.00","$1,396,118.36"
4,E,"$55,000.00","$989,900.00","$1,747,005.73"
5,L,"$12,100,000.00","$21,000,000.00","$16,187,727.27"
6,S,"$500,000.00","$9,985,000.00","$5,003,588.62"


In [15]:
import pandas as pd

# Supongamos que df_total ya está definido y contiene las columnas 'metros_construido' y 'precio_m2_construido'.

# Filtrar los datos según las condiciones especificadas
filtrado_df = df_total[(~df_total['metros_construido'].isin([0, 1, 2, 3, 4, 5])) & (df_total['precio_m2_construido'] >= 280000)]

# Paso 1: Calcular las estadísticas descriptivas
todo_descripcion = filtrado_df['precio_m2_construido'].describe()[['min', 'max', 'mean']].to_frame().T

# Agregar la columna de segmento
todo_descripcion['segmento'] = 'general'

# Paso 2: Renombrar las columnas
todo_descripcion.rename(columns={
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)

# Paso 3: Calcular el percentil 95%
percentil_95 = filtrado_df['precio_m2_construido'].quantile(0.95)

# Paso 4: Reemplazar el valor máximo con el percentil 95%
todo_descripcion['Máximo'] = percentil_95

# Paso 5: Aplicar formato monetario a los valores
def formato_monetario(valor):
    return f"${valor:,.2f}"  # Formato monetario con dos decimales

for i in range(3):  # 0, 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)

# Mostrar el resultado final
print(todo_descripcion)


                     Mínimo Máximo Promedio segmento
precio_m2_construido   $nan   $nan     $nan  general


C:\Users\yoe11\AppData\Local\Temp\ipykernel_2772\79234579.py:32: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'precio_m2_construido    $nan
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_2772\79234579.py:32: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'precio_m2_construido    $nan
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_2772\79234579.py:32: FutureWarning: Setting an item of incompatible dtype is deprecated and will r

In [16]:
import pandas as pd

# Supongamos que df_total ya está definido y contiene las columnas 'segmento' y 'precio'.

# Paso 1: Agrupar por segmento y calcular las estadísticas descriptivas
grupo_descripcion = df_total.groupby('segmento')['precio'].describe().reset_index()

# Paso 2: Seleccionar las columnas deseadas
grupo_descripcion = grupo_descripcion[['segmento', 'min', 'max', 'mean']]

# Paso 3: Renombrar las columnas
grupo_descripcion.rename(columns={
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)

# Paso 4: Calcular el percentil 95% para cada segmento
percentiles_95 = df_total.groupby('segmento')['precio'].quantile(0.95).reset_index()
percentiles_95.rename(columns={'precio': 'Máximo_95%'}, inplace=True)

# Paso 5: Unir los percentiles 95% con el DataFrame original
grupo_descripcion = grupo_descripcion.merge(percentiles_95, on='segmento')

# Paso 6: Reemplazar el valor máximo con el percentil 95%
grupo_descripcion['Máximo'] = grupo_descripcion['Máximo_95%']

# Eliminar la columna temporal
grupo_descripcion.drop(columns=['Máximo_95%'], inplace=True)

# Paso 7: Calcular las estadísticas generales
general_stats = df_total['precio'].describe()[['min', 'max', 'mean']].to_frame().T
general_stats.rename(columns={
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)

# Calcular el percentil 95% general
general_percentil_95 = df_total['precio'].quantile(0.95)

# Reemplazar el máximo general con el percentil 95%
general_stats['Máximo'] = general_percentil_95

# Agregar una fila para los resultados generales
general_stats['segmento'] = 'General'

# Concatenar los resultados generales al DataFrame de segmentos
resultado_final = pd.concat([grupo_descripcion, general_stats], ignore_index=True)

# Mostrar el resultado final
print(resultado_final)


  segmento      Mínimo      Máximo      Promedio
0        A   3255000.0   3950000.0  3.601984e+06
1        B   2500002.0   3200000.0  2.881962e+06
2        C   1750050.0   2450000.0  2.115324e+06
3        D   1005000.0   1700000.0  1.396118e+06
4        E     55000.0    989900.0  1.747006e+06
5        L  12100000.0  21000000.0  1.618773e+07
6        S    500000.0   9985000.0  5.003589e+06
7  General     55000.0   8500000.0  3.216404e+06


In [17]:
# Función para formatear valores en formato monetario
def formato_monetario(valor):
    return f"${valor:,.2f}"  # Formato con $ y 2 decimales, separador de miles

# Aplicar formato en m2 a los valores, excluyendo la columna 'segmento'
for i in range(1, 4):  # 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio'
    resultado_final.iloc[:, i] = resultado_final.iloc[:, i].astype(float).apply(formato_monetario)
resultado_final 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_2772\2150460987.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0     $3,255,000.00
1     $2,500,002.00
2     $1,750,050.00
3     $1,005,000.00
4        $55,000.00
5    $12,100,000.00
6       $500,000.00
7        $55,000.00
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  resultado_final.iloc[:, i] = resultado_final.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_2772\2150460987.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0     $3,950,000.00
1     $3,200,000.00
2     $2,450,000.00
3     $1,700,000.00
4       $989,900.00
5    $21,000,000.00
6     $9,985,000.00
7     $8,500,000.00
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a com

,segmento,Mínimo,Máximo,Promedio
0,A,"$3,255,000.00","$3,950,000.00","$3,601,983.91"
1,B,"$2,500,002.00","$3,200,000.00","$2,881,961.62"
2,C,"$1,750,050.00","$2,450,000.00","$2,115,324.05"
3,D,"$1,005,000.00","$1,700,000.00","$1,396,118.36"
4,E,"$55,000.00","$989,900.00","$1,747,005.73"
5,L,"$12,100,000.00","$21,000,000.00","$16,187,727.27"
6,S,"$500,000.00","$9,985,000.00","$5,003,588.62"
7,General,"$55,000.00","$8,500,000.00","$3,216,403.68"


In [18]:
# Guardar el JSON en un archivo
#resultados_finales_descripcion = pd.concat([grupo_descripcion, todo_descripcion], ignore_index=True)
resultados_finales_descripcion = resultado_final
resultados_finales_desc_json = resultados_finales_descripcion.to_json(orient='index')
with open('assets/tablas/grupo_precio_mercado.json', 'w') as f:
    f.write(resultados_finales_desc_json)

In [19]:
resultados_finales_descripcion

,segmento,Mínimo,Máximo,Promedio
0,A,"$3,255,000.00","$3,950,000.00","$3,601,983.91"
1,B,"$2,500,002.00","$3,200,000.00","$2,881,961.62"
2,C,"$1,750,050.00","$2,450,000.00","$2,115,324.05"
3,D,"$1,005,000.00","$1,700,000.00","$1,396,118.36"
4,E,"$55,000.00","$989,900.00","$1,747,005.73"
5,L,"$12,100,000.00","$21,000,000.00","$16,187,727.27"
6,S,"$500,000.00","$9,985,000.00","$5,003,588.62"
7,General,"$55,000.00","$8,500,000.00","$3,216,403.68"


## M2 Terreno

In [21]:
df_total = df_total[df_total['metros_construido'] > 40]
df_total_terreno = df_total[~df_total['metros_total'].isin([0, 1, 2])]

df_total_terreno = df_total_terreno[(df_total_terreno['metros_total'] != 1) & (df_total_terreno['metros_total'] > 20)]

df_descripcion = df_total_terreno.groupby('segmento')['metros_total'].describe(percentiles=[0.96]).reset_index()
df_descripcion = df_descripcion[['segmento', 'min', '96%', 'mean']] #Columnas deseadas
df_descripcion.rename(columns={
    'min': 'Mínimo',
    '96%': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)


# Función para formatear valores en formato metros
def formato_metros(valor):
    return f"{valor:,.2f} m²"

# Aplicar formato en m2 a los valores, excluyendo la columna 'segmento'
for i in range(1, 4):  # 1, 2, 3, 4 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    df_descripcion.iloc[:, i] = df_descripcion.iloc[:, i].astype(float).apply(formato_metros)
df_descripcion 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_2772\1393368157.py:21: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0     75.00 m²
1     62.00 m²
2     44.00 m²
3     45.00 m²
4     38.00 m²
5    245.00 m²
6     50.00 m²
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_descripcion.iloc[:, i] = df_descripcion.iloc[:, i].astype(float).apply(formato_metros)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_2772\1393368157.py:21: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0      318.00 m²
1      263.00 m²
2      247.00 m²
3      195.00 m²
4      653.00 m²
5    2,000.00 m²
6      500.00 m²
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_descripcion.iloc[:, i] = df_descripcion.iloc[:, i].astype(float).apply(fo

,segmento,Mínimo,Máximo,Promedio
0,A,75.00 m²,318.00 m²,163.25 m²
1,B,62.00 m²,263.00 m²,171.46 m²
2,C,44.00 m²,247.00 m²,149.22 m²
3,D,45.00 m²,195.00 m²,122.10 m²
4,E,38.00 m²,653.00 m²,175.20 m²
5,L,245.00 m²,"2,000.00 m²",666.88 m²
6,S,50.00 m²,500.00 m²,241.06 m²


In [22]:
todo_descripcion = df_total[~df_total['metros_total'].isin([0, 1, 2])]['metros_total'].describe(percentiles=[0.95])[['min', '95%', 'mean']].to_frame().T
todo_descripcion['segmento'] = 'general'

todo_descripcion.rename(columns={
    'min': 'Mínimo',
    '95%': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)

# Aplicar formato monetario a los valores
for i in range(3):  # 0, 1, 2 corresponden a 'Mínimo', 'Máximo', 'Promedio'
    todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_metros)

todo_descripcion 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_2772\1067183333.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'metros_total    38.00 m²
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_metros)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_2772\1067183333.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'metros_total    400.00 m²
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_metros)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_2772\1067183333.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in

,Mínimo,Máximo,Promedio,segmento
metros_total,38.00 m²,400.00 m²,186.87 m²,general


In [23]:
# Guardar el JSON en un archivo
resultados_finales_descripcion = pd.concat([df_descripcion, todo_descripcion], ignore_index=True)
resultados_finales_desc_json = resultados_finales_descripcion.to_json(orient='index')
with open('assets/tablas/grupo_terreno.json', 'w') as f:
    f.write(resultados_finales_desc_json)

In [24]:
resultados_finales_descripcion

,segmento,Mínimo,Máximo,Promedio
0,A,75.00 m²,318.00 m²,163.25 m²
1,B,62.00 m²,263.00 m²,171.46 m²
2,C,44.00 m²,247.00 m²,149.22 m²
3,D,45.00 m²,195.00 m²,122.10 m²
4,E,38.00 m²,653.00 m²,175.20 m²
5,L,245.00 m²,"2,000.00 m²",666.88 m²
6,S,50.00 m²,500.00 m²,241.06 m²
7,general,38.00 m²,400.00 m²,186.87 m²


## M2 Construcción

In [25]:
df_total_terreno = df_total[(df_total['metros_construido'] > 40) & (~df_total['metros_construido'].isin([0, 1,2,3]))]
df_descripcion = df_total_terreno.groupby('segmento')['metros_construido'].describe(percentiles=[0.95]).reset_index()
df_descripcion = df_descripcion[['segmento', 'min', '95%', 'mean']] #Columnas deseadas
df_descripcion.rename(columns={
    '95%': 'Máximo',
    'min': 'Mínimo',
    'mean': 'Promedio',
}, inplace=True)


# Función para formatear valores en formato metros
def formato_metros(valor):
    return f"{valor:,.2f} m²"

# Aplicar formato en m2 a los valores, excluyendo la columna 'segmento'
for i in range(1, 4):  # 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio'
    df_descripcion.iloc[:, i] = df_descripcion.iloc[:, i].astype(float).apply(formato_metros)
df_descripcion 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_2772\1810280376.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0     75.00 m²
1     62.00 m²
2     44.00 m²
3     45.00 m²
4     41.00 m²
5    245.00 m²
6     50.00 m²
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_descripcion.iloc[:, i] = df_descripcion.iloc[:, i].astype(float).apply(formato_metros)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_2772\1810280376.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0    290.00 m²
1    260.00 m²
2    250.00 m²
3    199.00 m²
4    409.00 m²
5    834.00 m²
6    500.00 m²
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_descripcion.iloc[:, i] = df_descripcion.iloc[:, i].astype(float).apply(formato_metros)


,segmento,Mínimo,Máximo,Promedio
0,A,75.00 m²,290.00 m²,216.10 m²
1,B,62.00 m²,260.00 m²,230.52 m²
2,C,44.00 m²,250.00 m²,159.53 m²
3,D,45.00 m²,199.00 m²,119.73 m²
4,E,41.00 m²,409.00 m²,192.79 m²
5,L,245.00 m²,834.00 m²,567.30 m²
6,S,50.00 m²,500.00 m²,267.92 m²


In [26]:
todo_descripcion = df_total[~df_total['metros_construido'].isin([0,1,2,3]) & (df_total['metros_total'] > 3)]['metros_total'].describe(percentiles=[0.95])[['min', '95%', 'mean',]].to_frame().T
todo_descripcion['segmento'] = 'general'

todo_descripcion.rename(columns={
    '95%': 'Máximo',
    'min': 'Mínimo',
    'mean': 'Promedio',
}, inplace=True)

# Aplicar formato monetario a los valores
for i in range(3):  # 0, 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Máximo Total', 'Promedio'
    todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_metros)

todo_descripcion 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_2772\2220166576.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'metros_total    38.00 m²
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_metros)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_2772\2220166576.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'metros_total    400.00 m²
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_metros)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_2772\2220166576.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in

,Mínimo,Máximo,Promedio,segmento
metros_total,38.00 m²,400.00 m²,186.87 m²,general


In [27]:
# Guardar el JSON en un archivo
resultados_finales_descripcion = pd.concat([df_descripcion, todo_descripcion], ignore_index=True)
resultados_finales_desc_json = resultados_finales_descripcion.to_json(orient='index')
with open('assets/tablas/grupo_construidos.json', 'w') as f:
    f.write(resultados_finales_desc_json)

In [28]:
resultados_finales_descripcion

,segmento,Mínimo,Máximo,Promedio
0,A,75.00 m²,290.00 m²,216.10 m²
1,B,62.00 m²,260.00 m²,230.52 m²
2,C,44.00 m²,250.00 m²,159.53 m²
3,D,45.00 m²,199.00 m²,119.73 m²
4,E,41.00 m²,409.00 m²,192.79 m²
5,L,245.00 m²,834.00 m²,567.30 m²
6,S,50.00 m²,500.00 m²,267.92 m²
7,general,38.00 m²,400.00 m²,186.87 m²


## Pecio M2 Construcción

In [29]:
df_total_terreno = df_total[~df_total['metros_construido'].isin([0, 1,2,3])]
df_total_terreno = df_total_terreno[df_total_terreno['precio_m2_construido'] > 10]
grupo_filtrado = df_total_terreno.groupby('segmento')['precio_m2_construido'].describe(percentiles=[0.95]).reset_index()
# Mantener solo las columnas que deseas (en este caso, min, 95%, mean)
grupo_filtrado = grupo_filtrado[['segmento', 'min', '95%', 'mean']] #Columnas deseadas
grupo_filtrado.rename(columns={
    '95%': 'Máximo',
    'min': 'Mínimo',
    'mean': 'Promedio',
}, inplace=True)

# Aplicar formato monetario a los valores, excluyendo la columna 'segmento'
for i in range(1, 4):  # 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio'
    grupo_filtrado.iloc[:, i] = grupo_filtrado.iloc[:, i].astype(float).apply(formato_monetario)

grupo_filtrado 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_2772\952371004.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0     $8,043.48
1       $107.25
2     $3,358.64
3     $2,400.66
4        $63.06
5    $17,375.00
6     $1,973.68
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_filtrado.iloc[:, i] = grupo_filtrado.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_2772\952371004.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0    $29,919.38
1    $26,616.67
2    $21,018.69
3    $19,200.00
4    $15,833.33
5    $42,500.00
6    $33,475.70
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_filtrado.iloc[:, i] = grupo_filtrado.iloc[:, i].astype(float).apply(f

,segmento,Mínimo,Máximo,Promedio
0,A,"$8,043.48","$29,919.38","$18,091.71"
1,B,$107.25,"$26,616.67","$16,086.00"
2,C,"$3,358.64","$21,018.69","$14,335.25"
3,D,"$2,400.66","$19,200.00","$12,598.32"
4,E,$63.06,"$15,833.33","$8,092.55"
5,L,"$17,375.00","$42,500.00","$30,200.68"
6,S,"$1,973.68","$33,475.70","$19,329.26"


In [30]:
todo_descripcion = df_total[(df_total['precio_m2_construido'] > 6) & (~df_total['metros_construido'].isin([0, 1,2,3]))]['precio_m2_construido'].describe(percentiles=[0.95])[['min', '95%', 'mean']].to_frame().T
todo_descripcion['segmento'] = 'general'
todo_descripcion = todo_descripcion[['segmento', 'min', '95%', 'mean']] #Columnas deseadas

todo_descripcion.rename(columns={
    '95%': 'Máximo',
    'min': 'Mínimo',
    'mean': 'Promedio',
}, inplace=True)

# Aplicar formato monetario a los valores, excluyendo la columna 'segmento'
for i in range(1, 4):  # 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio'
    todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)

todo_descripcion 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_2772\4241514475.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'precio_m2_construido    $63.06
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_2772\4241514475.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'precio_m2_construido    $28,767.87
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_2772\4241514475.py:13: FutureWarning: Setting an item of incompatible dtype is depreca

,segmento,Mínimo,Máximo,Promedio
precio_m2_construido,general,$63.06,"$28,767.87","$15,340.44"


In [31]:
# Guardar el JSON en un archivo
resultados_finales_descripcion = pd.concat([grupo_filtrado, todo_descripcion], ignore_index=True)
resultados_finales_desc_json = resultados_finales_descripcion.to_json(orient='index')
with open('assets/tablas/grupo_precio_construidos.json', 'w') as f:
    f.write(resultados_finales_desc_json)

In [32]:
resultados_finales_descripcion

,segmento,Mínimo,Máximo,Promedio
0,A,"$8,043.48","$29,919.38","$18,091.71"
1,B,$107.25,"$26,616.67","$16,086.00"
2,C,"$3,358.64","$21,018.69","$14,335.25"
3,D,"$2,400.66","$19,200.00","$12,598.32"
4,E,$63.06,"$15,833.33","$8,092.55"
5,L,"$17,375.00","$42,500.00","$30,200.68"
6,S,"$1,973.68","$33,475.70","$19,329.26"
7,general,$63.06,"$28,767.87","$15,340.44"
